In [1]:
import cv2
import numpy as np

import ipywidgets as widgets
from IPython.display import display, clear_output
import time

In [ ]:
# Hyper Parameters
lower_green = np.array([67, 23, 101])
upper_green = np.array([80, 255, 168])

lower_line = np.array([67, 23, 101])
upper_line = np.array([80, 255, 168])

ROI_X_start = 240
ROI_X_end = 480
ROI_Y_start = 150
ROI_Y_end = 490

ROI = [ROI_X_start, ROI_X_end, ROI_Y_start, ROI_Y_end]

rho = 1
theta = np.pi / 180
threshold = 80
min_line_length = 50
max_line_gap = 10
angle_threshold = 30

In [3]:
class Wrapper:
    def __init__(self, save_name=None):
        self.cap = cv2.VideoCapture(
            "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=640, height=480, framerate=15/1, format=NV12 ! "
            "nvvidconv flip-method=2 ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
        )
    def __enter__(self):
        print('entered')
        return self.cap
    def __exit__(self,a,b,c):
        self.cap.release()
        print('released')

In [ ]:
def detect_lines(image, ROI, rho, theta, threshold, min_line_length, max_line_gap):
    crop_img = image[ROI[0]:ROI[1], ROI[3]:ROI[4]].copy()
    gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)
    edges = cv2.Canny(binary, 50, 150, apertureSize=3)
    lines = cv2.HoughLineWsP(edges, rho, theta, threshold, minLineLength=min_line_length, maxLineGap=max_line_gap)

    if lines is None:
        return image, crop_img, [], None, 0

    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(crop_img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    image[240:480, 150:490] = crop_img

    if len(lines) > 0:
        center_line = np.mean(lines, axis=0)[0]
        center_x = int((center_line[0] + center_line[2]) / 2)
        center_y = int((center_line[1] + center_line[3]) / 2)
        cv2.circle(crop_img, (center_x, center_y), 5, (255, 0, 0), -1)
    else:
        center_x = None

    return image, crop_img, center_x

In [ ]:
def adjust_wheel_speed(center_x, frame_width):
    base_speed = 50
    max_speed_diff = 20
    if center_x is None:
        return base_speed, base_speed

    center_offset = center_x - (frame_width // 2)
    steering_angle = center_offset * (30 / (frame_width // 2))
    speed_diff = (steering_angle / 30) * max_speed_diff

    left_wheel_speed = base_speed - speed_diff
    right_wheel_speed = base_speed + speed_diff

    left_wheel_speed = max(0, min(100, left_wheel_speed))
    right_wheel_speed = max(0, min(100, right_wheel_speed))

    return left_wheel_speed, right_wheel_speed

In [ ]:
# 카메라에서 실시간 이미지 출력하는 함수
def main():
    video_widget = widgets.Image(format='jpeg')

    with Wrapper() as cap:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            result, crop_result, center_x = detect_lines(frame, ROI, rho, theta, threshold, min_line_length, max_line_gap)
            left_speed, right_speed = adjust_wheel_speed(center_x, crop_result.shape[1])
            
            _, buffer = cv2.imencode('.jpg', result)
            video_widget.value = buffer.tobytes()
            
            clear_output(wait=True)
            display(video_widget)
            time.sleep(0.03)  # 프레임 간 간격

if __name__ == '__main__':
    main()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

released


KeyboardInterrupt: 